# Run PncA WandB Sweep

In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd

import torch
from torch_geometric.data import Data

import wandb

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running locally'

False

In [2]:
full_dataset = torch.load('datasets/full_real_dataset.pth')

### Set Up Params and Sweep Config

In [3]:
seed = 42
np.random.seed(seed)
random.seed(seed)

# logging params (only used for wandb metrics)
n_samples = len(full_dataset)
# cutoff_distance = 6.3  

# gcn params
num_node_features = 12
batch_size = 256
# hidden_channels = 64
# learning_rate = 0.001
# wd = 5e-5
epochs = 1500

wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [6]:
# First sweep:

sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'Test Accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'hidden_channels': {
        'values': [32, 64, 128, 256, 320, 384]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 1e-10,
        'max': 1e-1
        },
    'dropout': {
          'values': [0.2, 0.4, 0.5, 0.6, 0.8]
        },
    'cutoff_distance': {
        'distribution': 'uniform',
        'min': 5.5,
        'max': 14 
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-10,
        'max': 1e-1
      },
    }

sweep_config['parameters'] = parameters_dict


In [7]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Test Accuracy'},
 'parameters': {'cutoff_distance': {'distribution': 'uniform',
                                    'max': 14,
                                    'min': 5.5},
                'dropout': {'values': [0.2, 0.4, 0.5, 0.6, 0.8]},
                'hidden_channels': {'values': [32, 64, 128, 256, 320, 384]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.1,
                                  'min': 1e-10},
                'weight_decay': {'distribution': 'log_uniform_values',
                                 'max': 0.1,
                                 'min': 1e-10}}}


### Define Training Loop

In [8]:
project = "pnca-singletons-sweep"

In [9]:
sweep_id = wandb.sweep(sweep_config, project=project)
# sweep_id = 'nc8eahq7'
# sweep_id = 'i8r2s0bc'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: i8r2s0bc
Sweep URL: https://wandb.ai/dylan-home/pnca-singletons-sweep/sweeps/i8r2s0bc


In [10]:
def sweep_run():

    with wandb.init() as run:
        config = run.config

        model = run_model.pnca_simpleGCN(
            # sequences=sequences_dict,
            sequences=None,
            self_loops = False,
            cutoff_distance = config.cutoff_distance,
            edge_weight_func = '1-(dist/cutoff)',
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = config.hidden_channels,
            learning_rate = config.learning_rate,
            wd = config.weight_decay,
            dropout = config.dropout,
            lr_scheduling=False,
            epochs = epochs,
            dataset = full_dataset,
            normalise_ews=True,
            wandb_params={
                'use_wandb': False,
                'sweep': True
            }
        )

        # os.makedirs(f'saved_models/carter_ds_aug/{project}/{sweep_id}', exist_ok=True)
        
        # torch.save(model, f'saved_models/carter_ds_aug/{project}/{sweep_id}/{run.name}')

In [11]:
wandb.agent(sweep_id, sweep_run, project = project, count=100)

wandb: Agent Starting Run: 0d3zqy3j with config:
wandb: 	cutoff_distance: 8.756811569791214
wandb: 	dropout: 0.6
wandb: 	hidden_channels: 32
wandb: 	learning_rate: 2.6167706326170646e-06
wandb: 	weight_decay: 0.0002639324904566719
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dylan-home. Use `wandb login --relogin` to force relogin


Adjusting edge index and attaching edge weights for cutoff distance 8.756811569791214
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 1.0754, Test Loss: 1.0975
Epoch: 020, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.9105, Test Loss: 0.9281
Epoch: 030, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.8839, Test Loss: 0.9022
Epoch: 040, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.8740, Test Loss: 0.8931
Epoch: 050, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.8682, Test Loss: 0.8863
Epoch: 060, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.8643, Test Loss: 0.8797
Epoch: 070, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.8572, Test Loss: 0.8735
Epoch: 080, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.8439, Test Loss: 0.8675
Epoch: 090, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.8418, Test Loss: 0.8614
Epoch: 100, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.8379, Test Loss:

Test Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test F1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Loss,█▆▆▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Sensitivity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Specificity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train F1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,█▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Sensitivity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Specificity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,0.515


wandb: Agent Starting Run: 8qri6whe with config:
wandb: 	cutoff_distance: 12.353278292838937
wandb: 	dropout: 0.5
wandb: 	hidden_channels: 256
wandb: 	learning_rate: 8.298650305704545e-06
wandb: 	weight_decay: 7.727779621645622e-07
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Adjusting edge index and attaching edge weights for cutoff distance 12.353278292838937
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.4698, Test Acc: 0.4850, Train Loss: 1.4859, Test Loss: 1.4424
Epoch: 020, Train Acc: 0.5539, Test Acc: 0.5650, Train Loss: 0.6985, Test Loss: 0.6958
Epoch: 030, Train Acc: 0.4957, Test Acc: 0.4700, Train Loss: 0.6890, Test Loss: 0.6926
Epoch: 040, Train Acc: 0.5280, Test Acc: 0.5150, Train Loss: 0.6937, Test Loss: 0.6997
Epoch: 050, Train Acc: 0.4935, Test Acc: 0.4600, Train Loss: 0.6832, Test Loss: 0.6859
Epoch: 060, Train Acc: 0.5000, Test Acc: 0.4700, Train Loss: 0.6815, Test Loss: 0.6850
Epoch: 070, Train Acc: 0.5280, Test Acc: 0.5100, Train Loss: 0.6833, Test Loss: 0.6905
Epoch: 080, Train Acc: 0.5043, Test Acc: 0.4950, Train Loss: 0.6779, Test Loss: 0.6805
Epoch: 090, Train Acc: 0.5259, Test Acc: 0.4800, Train Loss: 0.6756, Test Loss: 0.6795
Epoch: 100, Train Acc: 0.5323, Test Acc: 0.4850, Train Loss: 0.6751, Test Loss

...

Adjusting edge index and attaching edge weights for cutoff distance 10.97714630537332
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.4720, Test Acc: 0.5000, Train Loss: 0.7122, Test Loss: 0.7050
Epoch: 020, Train Acc: 0.5474, Test Acc: 0.5200, Train Loss: 0.6651, Test Loss: 0.6790
Epoch: 030, Train Acc: 0.4763, Test Acc: 0.5000, Train Loss: 0.7850, Test Loss: 0.7797
Epoch: 040, Train Acc: 0.6056, Test Acc: 0.5650, Train Loss: 0.6352, Test Loss: 0.6570
Epoch: 050, Train Acc: 0.5603, Test Acc: 0.5250, Train Loss: 0.6508, Test Loss: 0.6943
Epoch: 060, Train Acc: 0.5366, Test Acc: 0.5150, Train Loss: 0.7660, Test Loss: 0.8329
Epoch: 070, Train Acc: 0.6897, Test Acc: 0.6300, Train Loss: 0.5975, Test Loss: 0.6385
Epoch: 080, Train Acc: 0.6918, Test Acc: 0.6500, Train Loss: 0.5818, Test Loss: 0.6346
Epoch: 090, Train Acc: 0.5754, Test Acc: 0.5200, Train Loss: 0.6446, Test Loss: 0.7222
Epoch: 100, Train Acc: 0.6487, Test Acc: 0.6050, Train Loss: 0.6662, Test Loss:

Test Accuracy,▁▁▂▂▃▄▂▂▃▄▂▃▃▄▂▅▆▇▇▄▇▄▂▃▄▃▇███▂▃▄▃▃▂▂▇▃▃
Test F1,▁▁██▄▄███▄▂▃▃▅█▇▆█▇█▇▄███▃▇▇███▃████████
Test Loss,█▃▁▁▁▁▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▂▂▁▁▃▁▁▁▁▃▃▁▂▁▂▂▁▁▂
Test Sensitivity,▁▁██▂▂██▇▂▁▂▂▃█▅▄▆▄▇▅▂█▇▇▂▄▄▅▅█▂▇█▇██▆▇█
Test Specificity,██▁▁██▁▁▃█████▂▅▇▅▇▃▇█▁▂▃█▇▇▇▇▁█▃▁▂▁▁▅▂▂
Train Accuracy,▁▁▂▃▂▄▃▃▄▃▁▂▂▅▄▆▆▇▇▄▇▃▃▄▄▂▇▇██▃▂▅▃▅▃▄█▅▄
Train F1,▁▁▇▇▃▄██▇▄▂▃▃▅█▇▆█▇▇▇▄███▃▇▇███▃████████
Train Loss,█▃▂▂▂▂▂▂▁▂▂▂▂▂▁▁▁▁▁▁▁▂▂▁▁▃▁▁▁▁▃▃▁▂▁▂▂▁▁▁
Train Sensitivity,▁▁██▂▂██▇▂▁▂▂▃█▆▄▆▅▇▅▂█▇▇▂▅▄▅▅█▂▇█▇██▆██
Train Specificity,██▁▁██▁▁▃█████▂▆█▆▇▄▇█▁▃▃███▇▇▁█▄▁▃▁▂▆▃▂
Test Accuracy,0.54


wandb: Agent Starting Run: ooi6kwgl with config:
wandb: 	cutoff_distance: 13.182016812227989
wandb: 	dropout: 0.2
wandb: 	hidden_channels: 320
wandb: 	learning_rate: 1.844743754235364e-07
wandb: 	weight_decay: 3.465273747104558e-09
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Adjusting edge index and attaching edge weights for cutoff distance 13.182016812227989
Early stopping enabled. Patience: 20. Min Delta: 0.


wandb: Ctrl + C detected. Stopping sweep.
